In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

2023-07-17 16:31:04.391578: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


데이터 셋 로드 (TFDS에서 가져온 EMNIST)

In [2]:
(etfds_train, etfds_test), etfds_info = tfds.load("emnist", split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)
print(etfds_info)

def normalize_img_ascii_lable(image, label):
    ascii_Lb = label
    if ascii_Lb >= 10:
        ascii_Lb += 8
    if ascii_Lb >= 43:
        ascii_Lb += 6
    ascii_Lb += 16
    return tf.cast(image, tf.float32) / 255., ascii_Lb

etfds_train = etfds_train.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_train = etfds_train.cache()
etfds_train = etfds_train.shuffle(etfds_info.splits['train'].num_examples)
etfds_train = etfds_train.batch(128)
etfds_train = etfds_train.prefetch(tf.data.experimental.AUTOTUNE)

etfds_test = etfds_test.map(normalize_img_ascii_lable, num_parallel_calls=tf.data.experimental.AUTOTUNE)
etfds_test = etfds_test.batch(128)
etfds_test = etfds_test.cache()
etfds_test = etfds_test.prefetch(tf.data.experimental.AUTOTUNE)

tfds.core.DatasetInfo(
    name='emnist',
    full_name='emnist/byclass/3.0.0',
    description="""
    The EMNIST dataset is a set of handwritten character digits derived from the NIST Special Database 19 and converted to a 28x28 pixel image format and dataset structure that directly matches the MNIST dataset.
    
    Note: Like the original EMNIST data, images provided here are inverted horizontally and rotated 90 anti-clockwise. You can use `tf.transpose` within `ds.map` to convert the images to a human-friendlier format.
    """,
    config_description="""
    EMNIST ByClass
    """,
    homepage='https://www.nist.gov/itl/products-and-services/emnist-dataset',
    data_path='/Users/varzero/tensorflow_datasets/emnist/byclass/3.0.0',
    file_format=tfrecord,
    download_size=535.73 MiB,
    dataset_size=349.16 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=62),
    }),
    supe

직접 만든 데이터 셋 로드

In [3]:
datasetImgs = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255., validation_split=0.1)
trainFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='categorical', subset='training', shuffle=True)
testFG = datasetImgs.flow_from_directory(directory="../dataset/", target_size=(28,28), batch_size=10, class_mode='categorical', subset='validation', shuffle=True)

Found 1515 images belonging to 101 classes.
Found 101 images belonging to 101 classes.


모델 세팅

In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(95, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 512)               401920    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 95)                48735     
                                                                 
Total params: 450,655
Trainable params: 450,655
Non-trainable params: 0
_________________________________________________________________


훈련시키기

In [5]:
model.fit(etfds_train, epochs=5)
model.fit(trainFG, epochs=5)

2023-07-17 16:31:10.964336: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [4]
	 [[{{node Placeholder/_4}}]]
2023-07-17 16:31:10.967041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [4]
	 [[{{node Placeholder/_1}}]]


Epoch 1/5
5453/5453 [==============================] - 122s 11ms/step - loss: 0.6870 - accuracy: 0.7850
Epoch 2/5
5453/5453 [==============================] - 43s 8ms/step - loss: 0.5137 - accuracy: 0.8260
Epoch 3/5
3063/5453 [===============>..............] - ETA: 28s - loss: 0.4849 - accuracy: 0.8326

모델 저장하기

In [ ]:
model.save('spemnist.h5')

확인하기

In [ ]:
model.evaluate(etfds_test, verbose=2)
model.evaluate(testFG, verbose=2)

2023-07-17 00:02:49.099949: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]
2023-07-17 00:02:49.107930: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype string and shape [1]
	 [[{{node Placeholder/_2}}]]


909/909 - 14s - loss: 0.4431 - accuracy: 0.8448 - 14s/epoch - 15ms/step


[0.4430514872074127, 0.8448458313941956]